<a href="https://colab.research.google.com/github/InYoDream/Hate-Speech-Detection/blob/main/hatespeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 87% 70.0M/80.9M [00:00<00:00, 186MB/s]
100% 80.9M/80.9M [00:00<00:00, 182MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('extracted zip file')

extracted zip file


In [ ]:
#dependencies

import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing | Extracting data from the csv file for the  training and testing

In [ ]:
tw_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
tw_data.shape

(1599999, 6)

In [ ]:
tw_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
c_name=['target','id','date','flag','user','text'] #column name
tw_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=c_name,encoding='ISO-8859-1')

In [ ]:
tw_data.shape

(1600000, 6)

In [ ]:
tw_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
tw_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
tw_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
tw_data.replace({'target':{4:1}},inplace=True)

In [ ]:
tw_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

STEMMING of Data ...

In [ ]:
stemobj=PorterStemmer()


In [ ]:
def stmfun(content):
  stemmed=re.sub('[^a-zA-Z]',' ',content)
  stemmed=stemmed.lower()
  stemmed=stemmed.split()
  stemmed=[stemobj.stem(word) for word in stemmed if not word in stopwords.words('english')]
  stemmed=' '.join(stemmed)
  return stemmed


In [ ]:
tw_data['stemmed']=tw_data['text'].apply(stmfun)

In [ ]:
 tw_data.head()

,target,id,date,flag,user,text,stemmed
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [ ]:
x=tw_data['stemmed'].values
y=tw_data['target'].values

In [ ]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [ ]:
print(y)

[0 0 0 ... 1 1 1]


SPLITTING DATA INTO SPLIT & TEST

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, stratify=y,random_state=2)

Now converting text data into numeric format(vectorize)

In [ ]:
vectorizer=TfidfVectorizer()

x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

Training the model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

Model Evaluation & Accuracy score

In [ ]:
#Accuracy on training data
x_train_pred=model.predict(x_train)
training_data_acc=accuracy_score(y_train,x_train_pred)

In [ ]:
print('Accuracy score is : ',training_data_acc)

Accuracy score is :  0.81018984375


In [ ]:
#Now accuracy of test data
x_test_pred=model.predict(x_test)
test_dt_acc=accuracy_score(y_test,x_test_pred)

In [ ]:
print("Accuracy = ",test_dt_acc)

Accuracy =  0.7780375


Saving the trained model using PICKLE

Naive Baye's : 73.6 % ,
Logistic Regression : 77.8 % ,
Support Vector Machine : 68.32 % ,
Random Forest : 75.2 %  

In [ ]:
import pickle

In [ ]:
filename='RndFor_model.sav'
pickle.dump(model,open(filename,'wb'))

Using save model for future preictions


In [ ]:
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [ ]:
x_new=x_test[200]
print(y_test[200])
prediction=loaded_model.predict(x_new)

print(prediction)

if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

1
[1]
Positive Tweet


In [ ]:
!pip install streamlit
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.59s

3 packages are looking for funding
  run `npm fund` for details

found 2 moderate severity vulnerabilities
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
import pickle

In [ ]:
pickle.dump(vectorizer,open("vectorizer.pkl","wb"))